In [36]:
!pip install fastapi uvicorn pyngrok pydantic

In [37]:
%%writefile app.py
from fastapi import FastAPI
import joblib
import pandas as pd
from pydantic import BaseModel
from typing import Dict, List

app = FastAPI()
model = joblib.load('lgbm_best_model.joblib')
threshold = 0.22272402875679262

class InputData(BaseModel):
    features: Dict[str, float]

class BatchInputData(BaseModel):
    rows: List[Dict[str, float]]

@app.post("/predict")
def predict(data: InputData):
    df = pd.DataFrame([data.features])
    proba = float(model.predict_proba(df)[:, 1][0])
    prediction = int(proba >= threshold)
    return {"probability": round(proba, 4), "prediction": prediction, "threshold_used": threshold}

@app.post("/predict_batch")
def predict_batch(data: BatchInputData):
    df = pd.DataFrame(data.rows)
    probas = model.predict_proba(df)[:, 1]
    predictions = (probas >= threshold).astype(int).tolist()
    return {"probabilities": [round(float(p), 4) for p in probas], "predictions": predictions}

@app.get("/health")
def health():
    return {"status": "ok"}

Overwriting app.py


In [38]:
import subprocess
import threading
from pyngrok import ngrok
import time

ngrok.set_auth_token("3A1Dh2EuJzEjVzlQxl2anFPxWQ3_7ddiaZnMyqxfR9oKTfCXM")

threading.Thread(target=lambda: subprocess.run(["uvicorn", "app:app", "--port", "8000"])).start()

time.sleep(3)

public_url = ngrok.connect(8000)
print(f"API URL: {public_url}")

API URL: NgrokTunnel: "https://nonboiling-patt-nikolai.ngrok-free.dev" -> "http://localhost:8000"
